In [18]:
import pandas as pd
import boto3
import sagemaker
from sagemaker.serializers import CSVSerializer
import json
import joblib
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import pickle

In [17]:
!pip install xgboost


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [19]:
# Set SageMaker and S3 client variables
sess = sagemaker.Session()
region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sagemaker_role = sagemaker.get_execution_role()

In [20]:
# Set read and write S3 buckets and locations
write_bucket = "sagemaker-filesyash"
read_bucket = "sagemaker-filesyash"

train_data_key = "train.csv"
test_data_key = "test.csv"
model_key = "model"
output_key = "output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"

In [21]:
#MODEL PARAMETERS
hyperparams = {
                "max_depth": 3,
                "eta": 0.2,
                "objective": "binary:logistic",
                "subsample" : 0.8,
                "colsample_bytree" : 0.8,
                "min_child_weight" : 3
              }

num_boost_round = 100
nfold = 3
early_stopping_rounds = 10

In [22]:
# Load and preprocess the training data
label_col = "LABEL_ENCODED"
data = pd.read_csv(train_data_uri)
del data['Unnamed: 0']

In [23]:
# Recolumnstraining data and target
train_features = data.drop(label_col, axis=1)
train_label = pd.DataFrame(data[label_col])
dtrain = xgb.DMatrix(train_features, label=train_label)

test_features = data.drop(label_col, axis=1)
test_label = pd.DataFrame(data[label_col])
dtest = xgb.DMatrix(test_features, label=test_label)

In [24]:
#TRAIN THE MODEL
model = (xgb.train(params=hyperparams, dtrain=dtrain, evals = [(dtrain,'train'), (dtest,'eval')], num_boost_round=num_boost_round, 
                  early_stopping_rounds=early_stopping_rounds, verbose_eval = 0)
        )

In [25]:
#SAVE THE TRAINED MODEL LOCALLY
with open("./xgboost-model", "wb") as f:
    joblib.dump(model, f)    

In [32]:
#SAVE THE TRAINED MODEL ON AMAZON S3
s3_client.upload_file(Filename="./xgboost-model", Bucket=write_bucket, Key = "model")

In [29]:
#LOAD LOCALLY SAVED MODEL
model_path = "./xgboost-model"
model = xgb.Booster()
model.load_model(model_path)

XGBoostError: basic_string::resize

In [35]:
# Define model artifacts path
model_artifacts_path = f"s3://{write_bucket}/xgboost-model/model.tar.gz"

# Set the XGBoost container image URI based on the desired version
xgboost_version = "1.4-1"
xgboost_image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/xgboost:{xgboost_version}-cpu-py3"

# Create the XGBoost model object
xgb_model = XGBoostModel(
    model_data=model_artifacts_path,
    image_uri=xgboost_image_uri,
    role=sagemaker_role,
    sagemaker_session=sess
)


NameError: name 'XGBoostModel' is not defined

In [28]:
#DEPLOY THE MODEL
endpoint_name = "xgboost-endpoint"
xgb_predictor = model.deploy(
    initial_instance_count= 1,
    instance_type = "ml.m4.xlarge",
    serializer=CSVSerializer(),
    endpoint_name = enpoint_name
    )

AttributeError: 'Booster' object has no attribute 'deploy'